<a href="https://colab.research.google.com/github/Mbaroudi/30-Days-Of-Python/blob/master/microsoft_Phi_3_mini_128k_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from getpass import getpass

# Prompt for the Hugging Face token securely
hugging_face_token = getpass('Please enter your Hugging Face token: ')
os.environ['HUGGING_FACE_HUB_TOKEN'] = hugging_face_token

# Now, you can use this environment variable with the Transformers library
from transformers import AutoModelForCausalLM, AutoTokenizer


model_name = "microsoft/Phi-3-mini-128k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.getenv('HUGGING_FACE_HUB_TOKEN'))
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=os.getenv('HUGGING_FACE_HUB_TOKEN'))

In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

# Use the installed accelerate package
from accelerate import Accelerator
accelerator = Accelerator()

model_name = "microsoft/Phi-3-mini-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", trust_remote_code=True).to("cuda")

#model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"cuda": accelerator.device}, torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=accelerator.device)

prompt = "tell me about data Mesh"
output = pipe(prompt, max_new_tokens=1000, return_full_text=True, temperature=0.0, do_sample=False)
print(output[0]['generated_text'])

# Adding audio communication to interact with the Phi-3-mini model
 This involves several steps. First, you need to transcribe audio input to text, use the model to generate a response, and then convert the text response back into audio. Here’s how you can approach this task using Python:

## Step 1: Speech Recognition
You'll need to transcribe the audio input to text. Python libraries like speech_recognition can be used for this purpose.

In [ ]:
!pip install SpeechRecognition pydub

import speech_recognition as sr

# Initialize the recognizer
r = sr.Recognizer()

# Convert your audio file to text
with sr.AudioFile('/content/test.wav') as source:
    audio_data = r.record(source)
    text = r.recognize_google(audio_data)

print(text)

## Step 2: Text Generation
Now, you can pass the transcribed text to your model and generate a response as you have in your code.

In [ ]:
response = pipe(text, max_new_tokens=1000, return_full_text=True, temperature=0.0, do_sample=False)
response_text = response[0]['generated_text']
print(response_text)


test here

In [ ]:
!pip install gTTS

from gtts import gTTS
from IPython.display import Audio

tts = gTTS(response_text)
tts.save('/content/response.mp3')
Audio('/content/response.mp3')


In [ ]:
def audio_to_audio_response(audio_path):
    # Speech recognition
    with sr.AudioFile(audio_path) as source:
        audio_data = r.record(source)
        input_text = r.recognize_google(audio_data)

    # Generate response
    response = pipe(input_text, max_new_tokens=1000, return_full_text=True, temperature=0.0, do_sample=False)
    response_text = response[0]['generated_text']

    # Text-to-Speech
    tts = gTTS(response_text)
    tts.save('response.mp3')
    return Audio('response.mp3')

# Call the function with your audio file
audio_response = audio_to_audio_response('/content/question.wav')
audio_response


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


**1. Text Extraction from PDF**
Before you can train or fine-tune the model, you need to extract text from your PDF files. This can be done using libraries such as PyMuPDF (also known as fitz), PyPDF2, or pdfplumber.

**Example using PyMuPDF:**

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 52.7 MB/s eta 0:00:00


In [ ]:
import fitz  # Install using pip install PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("/content/SQL-cheat-sheet.pdf")


**2. Data Preparation**
After extracting the text, you need to preprocess it to be suitable for training. This includes cleaning the text (removing unnecessary spaces, special characters, etc.) and possibly segmenting it into manageable pieces that fit the model’s input limitations.

**Example of Basic Text Cleaning:**

In [ ]:
import re

def clean_text(text):
    # Remove non-ASCII characters, multiple spaces, and strip lines
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned_text = clean_text(pdf_text)


**3. Model Fine-Tuning**
To fine-tune microsoft/Phi-3-mini-128k-instruct, you generally need a setup involving the Transformers library by Hugging Face, a suitable computing environment (preferably with GPUs), and an understanding of the specific capabilities and configuration options of the model.

**Setting up the Environment**

In [ ]:
!pip install datasets

**Fine-Tuning Code**
You can use the Trainer API from Hugging Face to fine-tune the model on your prepared data.

In [ ]:
!pip install flash-attention


In [ ]:
!nvidia-smi

In [ ]:
print(torch.cuda.get_device_name(0))


NVIDIA A100-SXM4-40GB


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
import torch
import os

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets,
    shuffle=True,
    batch_size=training_args.per_device_train_batch_size
)


# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct")




# Prepare dataset
def tokenize_function(examples):
    # Tokenize the texts and prepare `labels` for the model
    model_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    # For language modeling, the labels are usually the input_ids shifted by one token
    model_inputs["labels"] = model_inputs["input_ids"][:]
    return model_inputs

train_dataset = Dataset.from_dict({"text": [cleaned_text]})
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Detect if we have a GPU available and set our device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="/content/sample_data/results",                # Output directory to save model checkpoints
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir='/content/sample_data/logs',                  # Directory for storing logs
    learning_rate=2e-5,                    # You can adjust this according to your model's requirement
    per_device_train_batch_size=4,        # Increase the batch size if memory allows
    gradient_accumulation_steps=2,  # Accumulate gradients over 4 steps

    num_train_epochs=3,
    save_steps=1000,                       # Increase or decrease according to your preference
    save_total_limit=2,                    # Only keep a limited number of checkpoints
    warmup_steps=500,                      # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                     # Strength of weight decay
    load_best_model_at_end=True,           # Whether to load the best model (in terms of loss) at the end of training
    metric_for_best_model="loss",          # Use loss to determine the best model
    greater_is_better=False,               # Lower loss indicates a better model
    fp16=True,                             # Enable mixed precision training
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Start the training process
trainer.train()
model.save_pretrained('/content/sample_data/results/my_checkpoint')
tokenizer.save_pretrained('/content/sample_data/results/my_checkpoint')



In [ ]:
!nvidia-smi


**4. Considerations and Tips**

**Compute Resources:**
Training or fine-tuning a large model can be computationally expensive. Make
sure you have the necessary GPU resources.
Data Privacy and Security: Ensure that any sensitive information in your PDFs is handled securely.

**Regular Evaluation:**
Continuously evaluate the model on a validation set to monitor the fine-tuning progress and avoid overfitting.
Fine-tuning a model like Phi-3-mini might require substantial computational resources and domain-specific adjustments. It’s advisable to experiment with smaller subsets of your data first to fine-tune your approach before scaling up. If you encounter any specific issues or have further questions about the process, feel free to ask!






